In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import os
import random
from PIL import Image
import tensorflow as tf
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, UpSampling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import load_img, img_to_array
#from sklearn.decomposition import PCA
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor
!pip install umap-learn # Install umap-learn package
from umap import UMAP
from sklearn.mixture import GaussianMixture
import matplotlib.pyplot as plt

# Especifica o diretório onde suas imagens estão localizadas
image_directory = '/content/drive/MyDrive/geral3'

# Lista todos os arquivos de imagem no diretório
image_filenames = os.listdir(image_directory)
image_filenames = [filename for filename in image_filenames if filename.endswith(('.jpg', '.jpeg', '.png'))]

# Exibe o número total de imagens no diretório
print(f"Total de imagens no diretório: {len(image_filenames)}")

OSError: [Errno 5] Input/output error: '/content/drive/MyDrive/geral3'

In [ ]:
# Adiciona as barras de processo
pip install tqdm


Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/req_command.py", line 67, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 324, in run
    session = self.get_default_session(options)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/index_command.py", line 71, in get_default_session
    self._session = self.enter_context(self._build_session(options))
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/index_command.py", line 100, in _build_session
    session = PipSession(
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/network/session.py", line 344, in __init__
    self.headers["User-Agent"] = user_agent()
  File "/usr/local/lib/python3.10/dist-packages/pip/_i

In [ ]:
# Função para carregar e processar uma única imagem
def load_and_process_image(path):
    img = load_img(path)
    img = img_to_array(img)
    img /= 255.0
    return img

# Ordena os nomes dos arquivos pela sua numeração
image_filenames = sorted(image_filenames, key=lambda x: int(''.join(filter(str.isdigit, x))))

# Pergunta ao usuário o número de imagens a serem processadas
num_images_to_load = 36000

# Usa o número especificado de imagens
image_filenames = image_filenames[:num_images_to_load]
image_paths = [os.path.join(image_directory, filename) for filename in image_filenames]

# Carrega as imagens utilizando múltiplas threads
with ThreadPoolExecutor(max_workers=80) as executor:
    images = list(tqdm(executor.map(load_and_process_image, image_paths), total=len(image_paths)))

# Converte a lista de imagens em um array numpy
images = np.array(images)


In [ ]:
#Define uma semente para resultados serem reproduzíveis
n=12

np.random.seed(n)
tf.random.set_seed(n)
random.seed(n)

NameError: name 'np' is not defined

In [ ]:

# Função customizada de perda SSIM
def ssim_loss(y_true, y_pred):
    ssim_per_image = tf.image.ssim(y_true, y_pred, max_val=1.0)
    return 1 - tf.reduce_mean(ssim_per_image)

# Define a arquitetura do autoencoder
input_shape = (54, 54, 3)  # Ajuste conforme necessário

autoencoder = Sequential([
    # Codificador
    Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=input_shape),
    Conv2D(64, (3, 3), activation='relu', padding='same'),
    Conv2D(64, (5, 5), activation='relu', padding='same'),
    MaxPooling2D(2, 2, padding='same'),
    Conv2D(32, (3, 3), activation='relu', padding='same'),

    # Decodificador
    Conv2D(32, (3, 3), activation='relu', padding='same'),
    UpSampling2D((2, 2)),
    Conv2D(64, (5, 5), activation='relu', padding='same'),
    Conv2D(64, (3, 3), activation='relu', padding='same'),
    Conv2D(64, (3, 3), activation='relu', padding='same'),
    Conv2D(3, (3, 3), activation='linear', padding='same')
])

# Compila o autoencoder com perda SSIM
autoencoder.compile(optimizer=Adam(learning_rate=0.0002), loss=ssim_loss)

# Treina o autoencoder
autoencoder.fit(images, images, epochs=40, batch_size=40)

# Define o codificador (parte do autoencoder)
encoder = Model(inputs=autoencoder.input, outputs=autoencoder.layers[-1].output)

# Obtém os vetores de características
feature_vectors = encoder.predict(images)

# Inicializa listas para armazenar os vetores de características somados
summed_feature_vectors_first_4 = []
summed_feature_vectors_last_2 = []

# Itera pelos vetores de características e os soma conforme a especificação
for i in range(0, len(feature_vectors), 6):
    sum_first_4 = np.mean(feature_vectors[i:i+4], axis=0)
    summed_feature_vectors_first_4.append(sum_first_4)

    sum_last_2 = np.mean(feature_vectors[i+4:i+6], axis=0)
    summed_feature_vectors_last_2.append(sum_last_2)

# Converte as listas para arrays numpy
summed_feature_vectors_first_4 = np.array(summed_feature_vectors_first_4)
summed_feature_vectors_last_2 = np.array(summed_feature_vectors_last_2)

# Combina os dois arrays horizontalmente
combined_summed_features = np.hstack((summed_feature_vectors_first_4, summed_feature_vectors_last_2))

In [ ]:
# UMAP
!pip install umap-learn
flattened_features = combined_summed_features.reshape(len(combined_summed_features), -1)

# Aplica UMAP com o número de componentes especificado
n_components = 2
umap = UMAP(n_components=n_components, n_neighbors=5, min_dist=0.8, metric='euclidean')
umap_transformed_features = umap.fit_transform(flattened_features)

# Vizualização do UMAP
plt.figure(figsize=(10, 8))
plt.scatter(umap_transformed_features[:, 0], umap_transformed_features[:, 1], s=5, cmap='Spectral')
plt.title("Visualização UMAP das Características Reduzidas")
plt.show()

# Definindo o número ideal de clusters com BIC
n_components_range = range(1, 25)  # Exemplo de intervalo
bic_scores = []
for n_components in n_components_range:
    gmm = GaussianMixture(n_components=n_components, covariance_type='full')
    gmm.fit(umap_transformed_features)
    bic_scores.append(gmm.bic(umap_transformed_features))

# Número ótimo de clusters baseado no BIC
optimal_n_clusters = n_components_range[np.argmin(bic_scores)]
print(f"Número ótimo de clusters de acordo com o BIC: {optimal_n_clusters}")

# Plotando os scores do BIC
plt.figure(figsize=(20, 12))
plt.plot(n_components_range, bic_scores, marker='o')
plt.title('Pontuação BIC por Número de Clusters', fontsize=20)
plt.xlabel('Número de Clusters', fontsize=18)
plt.ylabel('Pontuação BIC', fontsize=18)
plt.xticks(n_components_range, fontsize=14)
plt.yticks(fontsize=14)
plt.show()

NameError: name 'combined_summed_features' is not defined

In [ ]:
# Ajuste do GMM com o número ótimo de clusters
gmm = GaussianMixture(n_components=optimal_n_clusters, n_init=1)
gmm.fit(umap_transformed_features)
cluster_labels = gmm.predict(umap_transformed_features)

# Mostrar número de ítens por grupo
cluster_counts = Counter(cluster_labels)
print("Number of items per cluster:")
for cluster_id in range(n_clusters):
    print(f"Cluster {cluster_id}: {cluster_counts[cluster_id]} items")


NameError: name 'GaussianMixture' is not defined

In [ ]:
# Exportação dos resultados UMAP e dos clusters em CSV
df = pd.DataFrame(umap_transformed_features, columns=['UMAP_1', 'UMAP_2'])
df['Cluster_ID'] = cluster_labels
save_path = '/content/drive/MyDrive/geral3/clusters9/location9.csv'
df.to_csv(save_path, index=False)
print(f'Arquivo salvo em: {save_path}')



In [ ]:
# Mostra mapa 2d UMAP
color_palette = ['red', 'blue', 'green', 'purple', 'orange', 'cyan',
                 'pink', 'lightgreen', 'lightblue', 'yellow', 'grey', 'brown']

# If you have more clusters than colors, consider generating a larger palette or repeating the palette.

# Map cluster labels to colors from the palette
colors = [color_palette[label] for label in cluster_labels]

plt.figure(figsize=(10, 7))
scatter = plt.scatter(umap_transformed_features[:, 0], umap_transformed_features[:, 1], c=colors, marker='.')
plt.title('Gaussian Mixture Clustering', fontsize=14)
plt.xlabel('UMAP 1', fontsize=12)
plt.ylabel('UMAP 2', fontsize=12)

# Create a custom legend placed outside the plot
from matplotlib.patches import Patch
legend_patches = [Patch(color=color_palette[i], label=f'Cluster {i}') for i in range(n_clusters)]
legend1 = plt.legend(handles=legend_patches, title="Cluster ID", fontsize=12, loc='upper left', bbox_to_anchor=(1, 1))
legend1.get_title().set_fontsize('14')

# Adjust layout to make room for the legend outside of the plot
plt.tight_layout()

plt.show()


In [ ]:
# Vizualização de exemplos de clusters
new_image_directory = '/content/drive/MyDrive/geralilus'
new_image_filenames = os.listdir(new_image_directory)
new_image_filenames = [filename for filename in new_image_filenames if filename.endswith(('.jpg', '.jpeg', '.png'))]
new_image_filenames = sorted(new_image_filenames, key=lambda x: int(''.join(filter(str.isdigit, x))))

clustered_groups = {}
for i, label in enumerate(cluster_labels):
    if label not in clustered_groups:
        clustered_groups[label] = []
    clustered_groups[label].append(i)

for cluster_id in sorted(clustered_groups.keys()):
    print(f'Cluster {cluster_id}')
    group_indices = clustered_groups[cluster_id]
    plt.figure(figsize=(10, 8))
    selected_indices = random.sample(group_indices, min(20, len(group_indices)))

    for i, idx in enumerate(selected_indices):
        if idx < len(new_image_filenames):
            img_filename = new_image_filenames[idx]
            img_full_path = os.path.join(new_image_directory, img_filename)
            img = Image.open(img_full_path)
            plt.subplot(4, 5, i + 1)
            plt.imshow(img)
            plt.axis('off')

    plt.show()